# Simulation command generation from simulation sheet
>'It's turtles all the way down!'

Plan: 
- Have array of sample settings, maybe optimized to give comparable $\sigma t$ to give comparable signal strength.

In [2]:
from definitions import s_t
import numpy as np
import pandas as pd
import os 


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 3e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(2.165, 0.02165), (4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.02

delta_rho = 2e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: sigma * t: {st}, permissible: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

R = 1000.0nm, t = 0.001m:
	 lambda = 2.165 Å: sigma * t: 0.055121766, permissible: False
	 Optimal t: t=0.0012033042843946863
	 lambda = 4.321 Å: sigma * t: 0.21957144215999996, permissible: True
	 Optimal t: t=0.0003020805280445737
	 lambda = 8 Å: sigma * t: 0.75264, permissible: True
	 Optimal t: t=8.812746756909194e-05
R = 300.0nm, t = 0.003m:
	 lambda = 2.165 Å: sigma * t: 0.0496095894, permissible: False
	 Optimal t: t=0.004011014281315621
	 lambda = 4.321 Å: sigma * t: 0.19761429794399998, permissible: True
	 Optimal t: t=0.0010069350934819121
	 lambda = 8 Å: sigma * t: 0.677376, permissible: True
	 Optimal t: t=0.0002937582252303065
R = 50.0nm, t = 0.01m:
	 lambda = 2.165 Å: sigma * t: 0.027560883, permissible: False
	 Optimal t: t=0.024066085687893726
	 lambda = 4.321 Å: sigma * t: 0.10978572107999997, permissible: True
	 Optimal t: t=0.006041610560891475
	 lambda = 8 Å: sigma * t: 0.37632, permissible: True
	 Optimal t: t=0.0017625493513818387


In [3]:
from definitions import s_t


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.015

delta_rho = 2.08e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: ")
        print(f"\t\ts*t: {st}\n\t\t0.1 <= s*t <= 0.8: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        # print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

R = 1000.0nm, t = 0.001m:
	 lambda = 4.321 Å: 
		s*t: 0.179025110787744
		0.1 <= s*t <= 0.8: True
	 lambda = 8 Å: 
		s*t: 0.613656576
		0.1 <= s*t <= 0.8: True
R = 300.0nm, t = 0.001m:
	 lambda = 4.321 Å: 
		s*t: 0.0537075332363232
		0.1 <= s*t <= 0.8: False
	 lambda = 8 Å: 
		s*t: 0.18409697279999998
		0.1 <= s*t <= 0.8: True
R = 50.0nm, t = 0.01m:
	 lambda = 4.321 Å: 
		s*t: 0.08951255539387198
		0.1 <= s*t <= 0.8: False
	 lambda = 8 Å: 
		s*t: 0.30682828799999995
		0.1 <= s*t <= 0.8: True


In [4]:
print("{:,.3e}".format(np.sqrt(6.1e-7) * 1e18))

7.810e+14


In [5]:
from definitions import *


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
Rs = [2e-6, 300e-9, 50e-9]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]

# Source sample (thickness) pairing (L0, DL, R, t)
source_sample = [(4.321, 0.04321, 2e-6, 1e-3), (8, 0.8, 2e-6, 1e-3), (4.321, 0.04321, 300e-9, 10e-3), (8, 0.8, 300e-9, 5e-3), (4.321, 0.04321, 50e-9, 10e-3), (8, 0.8,50e-9, 10e-3)]

thickness_map = {
    (4.321, 2e-6):  1e-3,
    (8, 2e-6):  1e-3,
    (4.321, 300e-9): 10e-3,
    (8, 300e-9): 5e-3,
    (4.321, 50e-9): 10e-3,
    (8, 50e-9): 10e-3,
}
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
print(f"Constant parameters: drho = {delta_rho * 1e-14}e14 m^-2; phi = {phi}")
for (L0, DL, R, t) in source_sample:
    # print(thickness_map[(L0, R)])
    print(F"R = {R * 1e9}nm, t = {round(t * 1e3,2)}mm, lambda = {L0} Å:")
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    permissible = st < 0.8 and st >= 0.1
    print(f"\ts*t: {st}\n\t0.1 <= s*t <= 0.8: {permissible}")
    sigma = s_t(R,t, L0 * 1e-10) / t
    t_optimum = 0.38 / sigma

Constant parameters: drho = 1.8e14 m^-2; phi = 0.015
R = 2000.0nm, t = 1.0mm, lambda = 4.321 Å:
	s*t: 0.26814042111329994
	0.1 <= s*t <= 0.8: True
R = 2000.0nm, t = 1.0mm, lambda = 8 Å:
	s*t: 0.9191232
	0.1 <= s*t <= 0.8: False
R = 300.0nm, t = 10.0mm, lambda = 4.321 Å:
	s*t: 0.4022106316699499
	0.1 <= s*t <= 0.8: True
R = 300.0nm, t = 5.0mm, lambda = 8 Å:
	s*t: 0.6893424
	0.1 <= s*t <= 0.8: True
R = 50.0nm, t = 10.0mm, lambda = 4.321 Å:
	s*t: 0.06703510527832499
	0.1 <= s*t <= 0.8: False
R = 50.0nm, t = 10.0mm, lambda = 8 Å:
	s*t: 0.22978079999999998
	0.1 <= s*t <= 0.8: True


In [23]:
import pandas as pd
import os 
df = pd.read_csv('simulations.csv', sep=',', header=0)

class Instrument:
    def __init__(self, id: str,  prec_type: str, L0: float, DL: float, theta_0: float, By_min: float, By_max: float):
        self.id = id
        self.prec_type = prec_type
        self.L0 = L0
        self.DL = DL
        self.theta_0 = theta_0
        self.By_min = By_min 
        self.By_max = By_max

    def delta_min(self):
        return compute_z(self.By_min,self.theta_0,self.L0)
    
    def delta_max(self):
        # Due to the focussing condition, one component will have field By_max/2 and the other By_max, giving a delta of By_max/2
        return compute_z(self.By_max/2,self.theta_0,self.L0)
    
    def delta_range(self):
        return self.delta_min(), self.delta_max()

    def __str__(self):
        d_min, d_max = self.delta_range()
        return f"""Instrument ID {self.id}
    \tSource: L0 = {self.L0 * 1e10} Å; DL = {self.DL * 1e10} Å
    \tPrecession device: type = {self.prec_type}; theta_0 = {round(self.theta_0,2)} rad; By_min = {self.By_min * 1e3}mT; By_max = {self.By_max * 1e3}mT
    \tdelta range: {round(d_min * 1e9,2)} - {round(d_max * 1e9, 2)}nm"""
    

# print(df.values)
instrs = []
for r in df.values:
    # print(r)
    id = r[0]
    L0 = float(r[1]) * 1e-10
    DL = float(r[2]) * 1e-10
    prec = r[4]
    theta_0 = np.deg2rad(float(r[5]))
    By_min = float(r[6]) * 1e-3
    By_max = float(r[7]) * 1e-3
    instr = Instrument(id, prec, L0, DL, theta_0, By_min, By_max)
    instrs.append(instr)

for instr in instrs[3:]:
    print(str(instr))

Instrument ID 2a
    	Source: L0 = 4.321 Å; DL = 0.04321 Å
    	Precession device: type = foil; theta_0 = 0.19 rad; By_min = 0.3mT; By_max = 100.0mT
    	delta range: 76.35 - 12724.24nm
Instrument ID 2b
    	Source: L0 = 4.321 Å; DL = 0.04321 Å
    	Precession device: type = wsp; theta_0 = 0.79 rad; By_min = 0.1mT; By_max = 63.0mT
    	delta range: 4.95 - 1560.21nm
Instrument ID 2c
    	Source: L0 = 4.321 Å; DL = 0.04321 Å
    	Precession device: type = iso; theta_0 = 0.35 rad; By_min = 0.1mT; By_max = 15.0mT
    	delta range: 13.61 - 1020.63nm
Instrument ID 3a
    	Source: L0 = 8.0 Å; DL = 0.8 Å
    	Precession device: type = foil; theta_0 = 0.36 rad; By_min = 0.3mT; By_max = 100.0mT
    	delta range: 134.69 - 22448.55nm
Instrument ID 3b
    	Source: L0 = 8.0 Å; DL = 0.8 Å
    	Precession device: type = wsp; theta_0 = 0.79 rad; By_min = 0.1mT; By_max = 63.0mT
    	delta range: 16.98 - 5348.03nm
Instrument ID 3c
    	Source: L0 = 8.0 Å; DL = 0.8 Å
    	Precession device: type = iso; th

In [29]:
N = 1000000
N_steps = 30
def find_overlap(interval1, interval2):
    a1, a2 = interval1
    b1, b2 = interval2

    # Calculate the start and end of the overlap interval
    start = max(a1, b1)
    end = min(a2, b2)

    # Check if there is an actual overlap
    if start <= end:
        return (start, end)
    else:
        return None  # No overlap
instr: Instrument = instrs[3]
with open('simulate.sh', 'w') as f:
    f.write('rm -rf data\n')
    for instr in instrs[3:]:
        print("-------------------------\n"+str(instr))
        for R in Rs:
            t = thickness_map[(instr.L0 * 1e10, R)]
            mode = 'GPU'
            # if instr.prec_type == 'foil':
            #     mode = 'CPU'
            prec_type = instr.prec_type
            if instr.prec_type == 'wsp':
                prec_type = 'iwsp'
            By = 0.01
            
            d_min, d_max = instr.delta_range()

            # print(f"delta_y range: {round(d_min * 1e9,2)} - {round(d_max * 1e9, 2)}nm")
            # print(f"\tRange of interest for R = {R * 1e9}nm: 0 - {3 * R * 1e9}")
            print(f"\tR = {R * 1e9}nm:")
            print(f"\t\tRange of interest:  0 - {3 * R * 1e9}nm")

            

            overlap = find_overlap((d_min, d_max), (0.0, 3 * R))
            if overlap == None:
                print("No overlap!")
                f.write(f'echo "Skipping {instr.id}_{int(R * 1e10)} due to non-overlapping ranges!\n')

                raise Exception("No overlap!")
            else:
                (a,b) = overlap
                fraction = (b - a) / (3 * R) * 100
                print(f"\t\tOverlap: {round(a * 1e9,2)} - {round(b * 1e9, 2)}nm ({round(fraction,1)}% of range)")
                max_ratio = b / d_max
                By_min = instr.By_min
                By_max = instr.By_max * max_ratio
                sim_cmd = f"./full-simulation.sh {N} 4 {N_steps} {By_min},{By_max} {instr.L0 * 1e10} {instr.DL* 1e10} {R * 1e10} {t} {prec_type} {prec_type}_empty {mode}; rm -rf data_{instr.id}_{int(R * 1e10)}; mv data data_{instr.id}_{int(R * 1e10)}"
                # print(sim_cmd)
                f.write(f'{sim_cmd}\n')


-------------------------
Instrument ID 2a
    	Source: L0 = 4.321 Å; DL = 0.04321 Å
    	Precession device: type = foil; theta_0 = 0.19 rad; By_min = 0.3mT; By_max = 100.0mT
    	delta range: 76.35 - 12724.24nm
	R = 2000.0nm:
		Range of interest:  0 - 6000.0nm
		Overlap: 76.35 - 6000.0nm (98.7% of range)
	R = 300.0nm:
		Range of interest:  0 - 900.0nm
		Overlap: 76.35 - 900.0nm (91.5% of range)
	R = 50.0nm:
		Range of interest:  0 - 150.0nm
		Overlap: 76.35 - 150.0nm (49.1% of range)
-------------------------
Instrument ID 2b
    	Source: L0 = 4.321 Å; DL = 0.04321 Å
    	Precession device: type = wsp; theta_0 = 0.79 rad; By_min = 0.1mT; By_max = 63.0mT
    	delta range: 4.95 - 1560.21nm
	R = 2000.0nm:
		Range of interest:  0 - 6000.0nm
		Overlap: 4.95 - 1560.21nm (25.9% of range)
	R = 300.0nm:
		Range of interest:  0 - 900.0nm
		Overlap: 4.95 - 900.0nm (99.4% of range)
	R = 50.0nm:
		Range of interest:  0 - 150.0nm
		Overlap: 4.95 - 150.0nm (96.7% of range)
-------------------------


# Milk sample (from dairy paper)

In [ ]:
delta_rho = 2.0e14 # 1/m^2 (?)
phi = 0.036
R = 50e-9
t = 2e-3
for (L0, _) in sources:
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    print(st)